### SQLAlchemy

「ＯＲＭ」 - Object Relational Mapper
リレーショナルDBを持つことで、異なる枠組みのデータやプログラムの間でデータ共有できる

In [3]:
from sqlalchemy import create_engine
engine=create_engine('sqlite:///:memory:',echo=True)
engine

Engine(sqlite:///:memory:)

#### SQLAlchemyでテーブルを作成する
定義や関連付けはsqlalchemy.ext.declarativeの
declarative_base()で作ったクラスを継承することで
テーブルcreateの固有定義が可能になる。

In [8]:
from sqlalchemy.ext import declarative
from sqlalchemy import Column,Integer,Unicode,UnicodeText
Base=declarative.declarative_base()
class Book(Base):
    __tablename__='books'
    id=Column(Integer,primary_key=True)
    title=Column(Unicode(100),nullable=False)
    price=Column(Integer,nullable=False)
    memo=Column(UnicodeText)
    def __repr__(self):
        return "<Book('%s','%s','%s')>"%(self.title,self.price,self.memo)
    
Base.metadata.create_all(engine)

2020-03-24 10:55:35,761 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("books")
2020-03-24 10:55:35,761 INFO sqlalchemy.engine.base.Engine ()


In [24]:
from sqlalchemy.orm import sessionmaker
Session=sessionmaker(bind=engine)
session=Session()
dir(session)

['_Session__binds',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_bind',
 '_after_attach',
 '_autoflush',
 '_before_attach',
 '_bulk_save_mappings',
 '_close_impl',
 '_conditional_expire',
 '_connection_for_bind',
 '_contains_state',
 '_delete_impl',
 '_deleted',
 '_dirty_states',
 '_enable_transaction_accounting',
 '_expire_state',
 '_expunge_states',
 '_flush',
 '_flush_warning',
 '_flushing',
 '_identity_cls',
 '_is_clean',
 '_merge',
 '_new',
 '_query_cls',
 '_register_altered',
 '_register_persistent',
 '_remove_newly_deleted',
 '_save_impl',
 '_save_or_update_impl',
 '_save_or_update_state',
 '_update_impl',
 '_validate_persi

In [11]:
book=Book(title='やさしいPython',price='2580',memo='SBクリエイティブ')
session.add(book)
session.commit()

session.add_all([
    Book(title='すっきりわかるJava入門',price='2680',memo='インプレス'),
    Book(title='独習PHP',price='3520',memo='翔泳社'),
])
session.commit()

2020-03-24 11:01:50,714 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 11:01:50,717 INFO sqlalchemy.engine.base.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?)
2020-03-24 11:01:50,718 INFO sqlalchemy.engine.base.Engine ('やさしいPython', '2580', 'SBクリエイティブ')
2020-03-24 11:01:50,720 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-24 11:01:50,722 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 11:01:50,724 INFO sqlalchemy.engine.base.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?)
2020-03-24 11:01:50,725 INFO sqlalchemy.engine.base.Engine ('すっきりわかるJava入門', '2680', 'インプレス')
2020-03-24 11:01:50,726 INFO sqlalchemy.engine.base.Engine INSERT INTO books (title, price, memo) VALUES (?, ?, ?)
2020-03-24 11:01:50,728 INFO sqlalchemy.engine.base.Engine ('独習PHP', '3520', '翔泳社')
2020-03-24 11:01:50,729 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
query=session.query(Book)
for book in query.all():
    print(book.title)

2020-03-24 11:02:54,215 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books
2020-03-24 11:02:54,217 INFO sqlalchemy.engine.base.Engine ()
やさしいPython
すっきりわかるJava入門
独習PHP


In [15]:
book=query.get(1)
print(book)
print(book.title)

<Book('やさしいPython','2580','SBクリエイティブ')>
やさしいPython


In [18]:
book=query.filter(Book.title=='独習PHP').filter(Book.id==3).first()
print(book)
print(book.title)

2020-03-24 11:07:51,767 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.title = ? AND books.id = ?
 LIMIT ? OFFSET ?
2020-03-24 11:07:51,768 INFO sqlalchemy.engine.base.Engine ('独習PHP', 3, 1, 0)
<Book('独習PHP','3520','翔泳社')>
独習PHP


In [20]:
count=query.count()
print(count)

2020-03-24 11:08:27,281 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books) AS anon_1
2020-03-24 11:08:27,282 INFO sqlalchemy.engine.base.Engine ()
3


In [22]:
book=query.get(1)
book.price=2500
session.commit()
book=query.get(1)
print(book)

2020-03-24 11:10:09,660 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-24 11:10:09,660 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 11:10:09,660 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2020-03-24 11:10:09,660 INFO sqlalchemy.engine.base.Engine (1,)
<Book('やさしいPython','2500','SBクリエイティブ')>


In [23]:
book=query.get(1)
session.delete(book)
session.commit()
book=query.get(1)
print(book)
for book in query.all():
    print(book.title)

2020-03-24 11:11:41,393 INFO sqlalchemy.engine.base.Engine DELETE FROM books WHERE books.id = ?
2020-03-24 11:11:41,394 INFO sqlalchemy.engine.base.Engine (1,)
2020-03-24 11:11:41,396 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-24 11:11:41,398 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-24 11:11:41,399 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books 
WHERE books.id = ?
2020-03-24 11:11:41,400 INFO sqlalchemy.engine.base.Engine (1,)
None
2020-03-24 11:11:41,403 INFO sqlalchemy.engine.base.Engine SELECT books.id AS books_id, books.title AS books_title, books.price AS books_price, books.memo AS books_memo 
FROM books
2020-03-24 11:11:41,404 INFO sqlalchemy.engine.base.Engine ()
すっきりわかるJava入門
独習PHP
